In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number, lit
from pyspark.sql.window import Window



catalog = "fingrid_test_workspace"

schema = "fingrid_load_control"

table_name = "load_control"

table = ".".join([catalog, schema, table_name])

df = spark.read.table(table)


df = df.select("source_dataset_id","source_dateset_name").dropDuplicates()

df = df.withColumn("source_dataset_id", F.col("source_dataset_id").cast("bigint"))

df = df.withColumn(
    "generate_type",
    F.when(F.col("source_dataset_id") == 248, "Solar")
     .when(F.col("source_dataset_id") == 245, "Wind")
     .otherwise("Other")
)

df = df.withColumn(
    "update_frequency",
    F.when(F.col("source_dataset_id") == 248, "15 mins")
     .when(F.col("source_dataset_id") == 245, "15 mins")
     .otherwise("Other")
)

external_path = "abfss://gold@fingridtest.dfs.core.windows.net/gold/dim_generate_type/"
df.write.format("delta").mode("overwrite").save(external_path)



df.display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS fingrid_test_workspace.fingrid_gold.dim_generate_type
USING DELTA
LOCATION 'abfss://gold@fingridtest.dfs.core.windows.net/gold/dim_generate_type/'
